In [1]:
import pandas as pd


import numpy as np
from sklearn import preprocessing


from lib.preprop import *
from lib.geo_to_vector import vectorize_geo
from lib.eda_visualization import *

First, we will load the dataframe that we scraped from the web.

In [10]:
df = pd.read_csv('data/dataframes/df_complete.csv').iloc[:,3:]
df.shape

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18,88) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(13048, 2868)

# Removing duplicated rows
We will use the `drop_duplicates` method to remove duplicated rows based on 'company_name' column.  


In [ ]:
df_no_dups = remove_duplicates(df,['company_name'])
df_no_dups.shape

We will sorting all the columns by groups - categorial, numerical and binary columns:

In [ ]:
cat_cols = ['company_name','company_about', 'founded', 'business model','employees','product stage','status','geographical markets','fund_stage',]
num_cols = ['total_raised','total_rounds', 'investors','ipo_price']
tag_cols = [col for col in df_no_dups.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in df_no_dups.columns if col.startswith('targetmarket_')]
sector_list = [col for col in df_no_dups.columns if col.startswith("sector_")]
target_ind_list = [col  for col in df_no_dups.columns if col.startswith("target_industry_")]
technology_list = [col  for col in df_no_dups.columns if col.startswith("core_technology_")]


Changing target_ind_list and technology_list columns to have only 1 underscore:  
For example, 'core_technology_ai' will be changed to 'technology_ai'  
This will be used later on for the feature engineering.

In [ ]:

new_tech_cols = []
new_industry_cols = []
for col in target_ind_list + technology_list:
    if col in technology_list:
        new_tech_cols.append("technology" + col[15:])
    
    elif col in target_ind_list:
        new_industry_cols.append("industry" + col[15:])


d= {}
for i in range(len(new_tech_cols)):
    d[technology_list[i]] =  new_tech_cols[i]

for i in range(len(new_industry_cols)):
    d[target_ind_list[i]] =  new_industry_cols[i]

df_no_dups.rename(columns=d, inplace=True)


# Update the list with new columns names: 

target_ind_list = [col for col in df_no_dups.columns if col.startswith("industry_")]
technology_list = [col for col in df_no_dups.columns if col.startswith("technology_")]

# Repairing columns  
Before removing NaN values, we will first identify what values we expect to have in below columns and we will implement it

## Column 'founded'
We will change the string representation of founded - "month/year" to an integer "year"  
We will use REGEX to find the year

In [ ]:
# Fixing 'founded' column
import re

print(f'type of "founded" col is : {df_no_dups.founded.dtype}')
print('\nRunning script..')

founded_pattern = r"(\d{4})"
year_list=[]
df_founded = df_no_dups.copy()

for i, val in enumerate(df_founded.founded) :
    year = val.split('/')[-1]
    m = re.search(founded_pattern, year)
    if m:
        year_list.append(int(m.group(0)))
        
        
df_founded['founded'] = year_list


print('\nOperation succesfull!')
print(f'type of "founded" col is : {df_founded.founded.dtype}')
        

## Column 'geographical markets'
'geographical markets' column contain a string representing the geographical markets the company aims for.  
The script in lib.geo_to_vector.py calculates the **precentage** of the target geographic market and add it to the dataframe.  
The script uses Selenium to scrape the data from [WorldMeters](https://www.worldometers.info),  
Which shows updated data of countries and global population.

In [ ]:
# Fixing 'geographical markets' column

print("Geographical markets col:")
print(df_founded['geographical markets'].head())
print('----------------------------------------')

print('\nRunning the script...\n')
df_geo_market = vectorize_geo(df_founded, 'c')

print('\n----------------------------------------')
print('Operation succesfull!\n')
print("Geographical percentage col:")
print(df_geo_market['geo_market_per'].head())

num_cols.append('geo_market_per')
cat_cols.remove('geographical markets')

df_geo_market = df_geo_market.drop(['geographical markets'], axis=1)
df_geo_market = df_geo_market.dropna(subset=['company_name'])


## Checkpoint 1 : save the new df

In this notebook, we decided to use checkpoints to save the dataframe after few steps.  
This is very helpful with handling the big data we have, and preventing us from running all the code from the top if something goes wrong.

In [35]:
# Uncomment the desired line

# SAVE THE DATA
# df_geo_market.to_csv('data/dataframes/cp1.csv')

# LOAD THE DATA
df_geo_market = pd.read_csv('data/dataframes/cp1.csv').iloc[:,1:]

cat_cols = ['company_name','company_about', 'founded', 'business model','employees','product stage','status','funding stage']
num_cols = ['total_raised','total_rounds', 'investors','ipo_price','geo_market_per']
tag_cols = [col for col in df_geo_market.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in df_geo_market.columns if col.startswith('targetmarket_')]
sector_list = [col for col in df_geo_market.columns if col.startswith("sector_")]
target_ind_list = [col for col in df_geo_market.columns if col.startswith("industry_")]
technology_list = [col for col in df_geo_market.columns if col.startswith("technology_")]


df_geo_market.shape

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (15,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(10070, 2868)

# Handling NaN values

In this part of the cleaning process, we will handle the NaN values.  

First, we will check how many null values are in each column of the non-binary columns:

In [14]:
print("missing values:")
cols_to_check = num_cols + cat_cols
for col in cols_to_check:
    s = df_geo_market[col].isnull().sum()
    print(f'\t{col}: {s} missing values') 

missing values:
	total_raised: 5704 missing values
	total_rounds: 5704 missing values
	investors: 5704 missing values
	ipo_price: 9920 missing values
	geo_market_per: 2912 missing values
	company_name: 0 missing values
	company_about: 2 missing values
	founded: 0 missing values
	business model: 68 missing values
	employees: 32 missing values
	product stage: 163 missing values
	status: 0 missing values
	funding stage: 292 missing values


In the data scraped from the web, we have 2 columns that represents the same information:  
1. 'fund_stage' <-> 'funding stage'
2. 'raised' <-> 'total_raised'  

Also, we will remove 'products' column since it is not relevant for the analysis.  
But first, we will check which of the duplicated columns hold more data:

In [32]:
print("Total nan values in below columns:")
print(f"\n'fund_stage' <-> 'funding stage'")
print(f"{df_geo_market['fund_stage'].isnull().sum()} <-> {df_geo_market['funding stage'].isnull().sum()}")

print(f"\n'raised' <-> 'total_raised' ")
print(f"{df_geo_market['raised'].isnull().sum()} <-> {df_geo_market['total_raised'].isnull().sum()}")

Total nan values in below columns:

'fund_stage' <-> 'funding stage'
5769 <-> 292

'raised' <-> 'total_raised' 
6528 <-> 5704


We see 'fund_stage' has more nan values than 'funding stage', and 'raised' has more nan values than 'total_raised'.  
Therefore, we will remove 'fund_stage' and 'raised' columns, in addition to 'products' column.  
Also, we will replace nan values in the following way:  
1. nan in a **categorical** column will be replaced with 'na'.
2. nan in a **numerical** column (except 'geo_market_per' column) will be replaced with 0. 
3. nan in the **'geo_market_per'** column will be replaced with the median value of the column.

In [36]:
print(f'Dataframe shape before :{df_geo_market.shape}')

drop_cols = ['fund_stage','products','raised']
new_df = df_geo_market.drop(drop_cols, axis =1)

new_df = repair_categorical_missing_vals(new_df, cat_cols)

new_df = repair_numeric_missing_vals_zero(new_df, [col for col in num_cols if col != 'geo_market_per'])
new_df = repair_numeric_missing_vals_median(new_df, ['geo_market_per'])

print(f'Dataframe shape after :{new_df.shape}')

Dataframe shape before :(10070, 2868)
Dataframe shape after :(10070, 2865)


Next, we will convert all numerical columns to float type:

In [37]:
for col in num_cols:
    new_df[col] = conv_to_float(new_df[col])

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


21.5M converted to 21500000
4.69M converted to 4690000
24M converted to 24000000
72.5M converted to 72500000
36.5M converted to 36500000
0 converted to 0.0
3M converted to 3000000
14.5M converted to 14500000
4.825M converted to 4825000
38.85M converted to 38850000
600K converted to 600000
118.7M converted to 118700000
100K converted to 100000
950K converted to 950000
1.379M converted to 1379000
2.35M converted to 2350000
14.5M converted to 14500000
500K converted to 500000
5.247M converted to 5247000
0 converted to 0.0
190M converted to 190000000
550K converted to 550000
5.25M converted to 5250000
1.844M converted to 1844000
350K converted to 350000
6M converted to 6000000
92.4M converted to 92400000
0 converted to 0.0
50.7M converted to 50700000
13.7M converted to 13700000
1M converted to 1000000
4M converted to 4000000
2M converted to 2000000
1.351M converted to 1351000
36M converted to 36000000
0 converted to 0.0
0 converted to 0.0
1.02M converted to 1020000
350K converted to 350000

In [38]:
print("Checking missing values in categorical and numeric columns...\n")

for col in num_cols + cat_cols:
    s = new_df[col].isnull().sum()
    print(f'{col}: {s} missing values') 

Checking missing values in categorical and numeric columns...

total_raised: 0 missing values
total_rounds: 0 missing values
investors: 0 missing values
ipo_price: 0 missing values
geo_market_per: 0 missing values
company_name: 0 missing values
company_about: 0 missing values
founded: 0 missing values
business model: 0 missing values
employees: 0 missing values
product stage: 0 missing values
status: 0 missing values
funding stage: 0 missing values


<h3> Dealing with NaN in binary columns  </h3>

After handling the nan values in the categorical columns and numeric columns, we can move forward to the massive binary columns.  

The scraping process gave a 1 value for each instance (company) only if the specific column key was present in the comapny url,  
Therefore, we can replace all nan values in the binary columns with a 0 :

In [18]:
new_df[tag_cols] = new_df[tag_cols].fillna(0)
new_df[targetmarket_cols] = new_df[targetmarket_cols].fillna(0)
new_df[sector_list] = new_df[sector_list].fillna(0)
new_df[target_ind_list] = new_df[target_ind_list].fillna(0)
new_df[technology_list] = new_df[technology_list].fillna(0)

And we will now check if we missed any nan value in the binary columns:

In [20]:
bin_cols = tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list

print("Checking missing values in binary columns...\n")
are_missing = False

for col in bin_cols:
    s = new_df[col].isnull().sum()
    if s!=0:
        are_missing = True
        print(f'{col}: {s} missing values') 

if are_missing is False:
    print("No Nan values in binary columns!")


Checking missing values in binary columns...

No Nan values in binary columns!


## Checkpoint 2 : save the new df

In [21]:
# Uncomment the desired line

# SAVE THE DATA
# new_df.to_csv('data/dataframes/cp2.csv')

# LOAD THE DATA
new_df = pd.read_csv('data/dataframes/cp2.csv').iloc[:,1:]

cat_cols = ['company_name','company_about', 'founded', 'business model','employees','product stage','status','funding stage']
num_cols = ['total_raised','total_rounds', 'investors','ipo_price', 'geo_market_per']
tag_cols = [col for col in new_df.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in new_df.columns if col.startswith('targetmarket_')]
sector_list = [col for col in new_df.columns if col.startswith("sector_")]
target_ind_list = [col for col in new_df.columns if col.startswith("industry_")]
technology_list = [col for col in new_df.columns if col.startswith("technology_")]

bin_cols = tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list

print(f'Shape of the dataframe: {new_df.shape}')

(10070, 2865)

<h1> Categorical columns </h1>

First, let's explore the categorical columns:

In [40]:
cat_df = new_df.copy()
cat_df[cat_cols].describe(include='all')

,company_name,company_about,founded,business model,employees,product stage,status,funding stage
count,10070,10070,10070.000000,10070,10070,10070,10070,10070
unique,10070,10015,NaN,16,6,7,2,11
top,Tastewise,This company is a known business entity but la...,NaN,B2B,1-10,Released,active,Bootstrapped
freq,1,48,NaN,4743,6232,6674,6554,2810
mean,NaN,NaN,2011.537736,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,10.052327,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1901.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2010.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2014.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2017.000000,NaN,NaN,NaN,NaN,NaN


We see that ['employees', 'business model', 'status', 'product stage' and 'funding stage'] columns contains few unique values,  
Therefore we can encode them with sklearn's LabelEncoder object.  
Lets try it on 'employees' column first:

In [41]:
# Employees col
print("Value counts for 'employees' column:\n")
print(cat_df.employees.value_counts())

print("\nApplying LabelEncoder.\n")

le = preprocessing.LabelEncoder()
cat_df.employees = le.fit_transform(cat_df.employees)
print("New value counts for 'employees' column:\n")
print(cat_df.employees.value_counts())

Value counts for 'employees' column:

1-10       6232
11-50      2683
51-200      822
201-500     189
500+        112
na           32
Name: employees, dtype: int64

Applying LabelEncoder.

New value counts for 'employees' column:

0    6232
1    2683
4     822
2     189
3     112
5      32
Name: employees, dtype: int64


We will do the same for the rest of the categorical columns besides 'active',  
For the 'active' column, we will use the replace mapping to replace 'active' value with 1 and 'not_active' with 0:

In [42]:
cols = ['business model', 'product stage', 'funding stage']
print("Classes for each categorical column applied with LabelEncoder :\n")
for col in cols:
    le = preprocessing.LabelEncoder()
    cat_df[col] = le.fit_transform(cat_df[col])
    print(f'{col}:\n{le.classes_}')

replace_map = {'active' : 1, 'not_active' : 0}
cat_df.status.replace(replace_map, inplace= True)

business model :
['B2B' 'B2B, B2B2C' 'B2B, B2C' 'B2B, B2C, B2B2C' 'B2B, B2C, B2G'
 'B2B, B2C, B2G, B2B2C' 'B2B, B2G' 'B2B, B2G, B2B2C' 'B2B2C' 'B2C'
 'B2C, B2B2C' 'B2C, B2G' 'B2C, B2G, B2B2C' 'B2G' 'B2G, B2B2C' 'na']
product stage :
['Alpha' 'Beta' 'Clinical Trial' 'Customer development' 'R&D' 'Released'
 'na']
funding stage :
['Acquired' 'Bootstrapped' 'Established' 'Pre-Seed' 'Public' 'ROUND A'
 'ROUND B' 'ROUND C+' 'Revenue Financed' 'Seed' 'na']


# 'succeeded' column:
After cleaning all the data (categorical, numerical and binary), we can move on to the research part of the project.  
In order to check if a company is successful, we will use the following condition:  
A company is successful if it has **Product success** or **Funding success**:  
1. Product success: 
    - if 'status' is **active** and 'product stage' is **released**.  
<br>
2. Funding success:
    - if 'funding stage' is **acquired** or
    - if 'status' is **active** and 'funding stage' is **public** or
    - if 'status' is **active** and 'total raised' is greater than a million dollars.  

We will use mask method to add a 'succeeded' column to the dataframe that will hold the success status of each company acording to the above condition.

In [44]:
# add succeeded column
success_rate = 4000000

PRODUCT_SUCCESS = (cat_df['status'] == 1) & (cat_df['product stage'] == 5)
FUNDING_SUCCESS = (cat_df['funding stage'] == 1) | ((cat_df['funding stage'] == 7) &
                  (cat_df['status'] == 1)) | ((cat_df['total_raised'] >= success_rate) &
                  (cat_df['status'] == 1))

cat_df.loc[PRODUCT_SUCCESS | FUNDING_SUCCESS , 'succeeded'] = 1
cat_df.loc[~PRODUCT_SUCCESS & ~FUNDING_SUCCESS, 'succeeded'] = 0

# Add the new succeeded column to categorical columns list: 
cat_cols.append('succeeded')

print(f'Total succeeded companies: {cat_df.succeeded.sum()}')

Total succeeded companies: 7009.0


# Save the cleaned dataframe
We will now save the cleaned dataframe as csv file and change columns order in the following way:  
cat_cols, num_cols, tag_cols, targetmarket_cols, sector_list, target_ind_list, technology_list

In [27]:
final_df = pd.DataFrame(cat_df, columns = cat_cols + num_cols + tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list)
final_df.to_csv('data/dataframes/final_cleaned.csv')

In this notebook we took the raw scraped dataframe and cleaned it.  
Next step is to detect the outliers in the dataframe.  
It is complicated to find relationship between binary columns, and we will tackle it in the next notebook: [Outlier detection notebook](outlier_detection.ipynb)